Nhiệm vụ 5 – Pivot Table + Stack/Unstack: 
Mục tiêu: rèn luyện khả năng chuyển đổi cấu trúc dữ liệu và phân tích theo nhiều chiều. 
- Pivot chi phí trung bình theo khoa × nhóm tuổi.
- Pivot số lượt khám theo tháng × khoa.
- Thực hiện stack/unstack.
- Nhận xét khoa nào có mật độ bệnh nhân cao nhất.

In [2]:
import pandas as pd

# 1. Đọc dữ liệu
base_path = r"D:\OneDrive\Máy tính\18A2_Nhom_4_Case_9\data_clean"
df_p = pd.read_csv(f"{base_path}\\patient_clean.csv")
df_v = pd.read_csv(f"{base_path}\\visit_clean.csv")
df_b = pd.read_csv(f"{base_path}\\billing_clean.csv")

# 2. Gộp dữ liệu
df_full = pd.merge(df_v, df_p, on='patient_id', how='inner')
df_full = pd.merge(df_full, df_b, on='visit_id', how='inner')

# BƯỚC QUAN TRỌNG: LÀM SẠCH DỮ LIỆU (FIX LỖI TYPE ERROR)
df_full['total_fee'] = pd.to_numeric(df_full['total_fee'], errors='coerce').fillna(0)
# Xử lý ngày tháng và tạo cột Tháng
df_full['visit_date'] = pd.to_datetime(df_full['visit_date'], format='mixed', dayfirst=True, errors='coerce')
df_full['month'] = df_full['visit_date'].dt.month

# Tạo nhóm tuổi (Age Group)
bins = [0, 18, 40, 60, 150]
labels = ['0-18', '19-40', '41-60', '>60']
df_full['age_group'] = pd.cut(df_full['age'], bins=bins, labels=labels)
#  PIVOT CHI PHÍ TRUNG BÌNH (KHOA x NHÓM TUỔI)
print("\n 1. PIVOT: CHI PHÍ TRUNG BÌNH THEO KHOA & NHÓM TUỔI")
pivot_cost = df_full.pivot_table(
    index='department',       # Hàng: Tên khoa
    columns='age_group',      # Cột: Nhóm tuổi
    values='total_fee',       # Giá trị: Chi phí
    aggfunc='mean'            # Hàm: Tính trung bình
)
print(pivot_cost.round(0).fillna(0))
#  PIVOT SỐ LƯỢT KHÁM (THÁNG x KHOA)
print("\n 2. PIVOT: SỐ LƯỢT KHÁM THEO THÁNG & KHOA ")
pivot_visits = df_full.pivot_table(
    index='month',            # Hàng: Tháng
    columns='department',     # Cột: Tên khoa
    values='visit_id',        # Giá trị: Mã lượt khám (để đếm)
    aggfunc='count',          # Hàm: Đếm số lượng
    fill_value=0              # Điền 0 nếu tháng đó khoa không có khách
)
print(pivot_visits)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\OneDrive\\Máy tính\\18A2_Nhom_4_Case_9\\data_clean\\patient_clean.csv'

In [5]:
# --- 2. STACK / UNSTACK ---
print("\n=== 2. STACK & UNSTACK ===")
stacked_df = pivot_cost.stack()
print(stacked_df.head())
print("\nUnstack check:")
print(stacked_df.unstack().iloc[:2])


=== 2. STACK & UNSTACK ===
department  age_group
Ngoai       0-18              0.000000
            19-40        128125.000000
Nhi         19-40        119696.969697
            41-60        187500.000000
Noi         0-18         120000.000000
dtype: float64

Unstack check:
age_group   0-18          19-40     41-60
department                               
Ngoai        0.0  128125.000000       NaN
Nhi          NaN  119696.969697  187500.0


In [6]:
# --- 4. NHẬN XÉT ---
print("\n=== 4. KẾT LUẬN ===")
dept_counts = df_full['department'].value_counts()
print(f"Khoa đông nhất: {dept_counts.idxmax()} ({dept_counts.max()} lượt).")


=== 4. KẾT LUẬN ===
Khoa đông nhất: Nhi (37 lượt).
